In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("/usr/local/lib/python3.6/site-packages/libsvm/")
from svmutil import *

In [2]:
X_train = pd.read_csv('X_train.csv',header=None).values
Y_train = pd.read_csv('Y_train.csv',header=None).values.reshape(-1)
X_test = pd.read_csv('X_test.csv',header=None).values
Y_test = pd.read_csv('Y_test.csv',header=None).values.reshape(-1)

In [35]:
kernels = ['linear','polynomial','RBF']
for i in range(3):
    print('Kernel: '+kernels[i])
    params = '-t '+ str(i)
    model = svm_train(Y_train,X_train,params)
    svm_predict(Y_test,X_test, model)

Kernel: linear
Accuracy = 95.08% (2377/2500) (classification)
Kernel: polynomial
Accuracy = 34.68% (867/2500) (classification)
Kernel: RBF
Accuracy = 95.32% (2383/2500) (classification)


In [14]:
c_begin,c_end,c_step = (-5,5,1) # Cost
g_begin,g_end,g_step = (-5,5,1) # Gamma
d_begin,d_end,d_step = (2,4,1)  # degree(poly. kernel)
r_begin,r_end,r_step = (0,2,1)  # coef.(poly. kernel)
best_acc = [0,0,0]
best_params = [0,(0,0,0,0),(0,0)]
RBF_res = []
for c in range(c_begin,c_end+c_step,c_step):
    params = '-v 5 -s 0 -t {} -c {}'.format(k,2**c)
        
    # Linear kernel
    params = '-v 5 -s 0 -t 0 -c {}'.format(2**c)
    acc = svm_train(Y_train,X_train,params)
    if acc > best_acc[0]:
        best_acc[0] = acc
        best_params[0] = 2**c
        
    # Polynomial kernel
    for g in range(g_begin,g_end+g_step,g_step):
        for d in range(d_begin,d_end+d_step,d_step):
            for r in range(r_begin,r_end+r_step,r_step):
                params = '-v 5 -s 0 -t 1 -c {} -g {} -d {} -r {}'.format(2**c,2**g,d,r)
                acc = svm_train(Y_train,X_train,params)
                if acc > best_acc[1]:
                    best_acc[1] = acc
                    best_params[1] = (2**c,2**g,d,r)
        
    # RBF kernel
    for g in range(g_begin,g_end+g_step,g_step):
        params = '-v 5 -s 0 -t 1 -c {} -g {}'.format(2**c,2**g)
        acc = svm_train(Y_train,X_train,params)
        if acc > best_acc[2]:
            best_acc[2] = acc
            best_params[2] = (2**c,2**g) 
        RBF_res.append((c,g,acc))
        
print("Best parameters of linear, polynomial, RBF kernel respectively: {}".format(best_params))
print("Best accuracy of linear, polynomial, RBF kernel respectively: {}".format(best_acc))

Cross Validation Accuracy = 96.94%
Cross Validation Accuracy = 96.72%
Cross Validation Accuracy = 97.16%
Cross Validation Accuracy = 97.16%
Cross Validation Accuracy = 96.44%
Cross Validation Accuracy = 97.86%
Cross Validation Accuracy = 97.94%
Cross Validation Accuracy = 95.42%
Cross Validation Accuracy = 98.02%
Cross Validation Accuracy = 98.1%
Cross Validation Accuracy = 97.7%
Cross Validation Accuracy = 97.76%
Cross Validation Accuracy = 97.82%
Cross Validation Accuracy = 97.64%
Cross Validation Accuracy = 98.1%
Cross Validation Accuracy = 98.04%
Cross Validation Accuracy = 96.82%
Cross Validation Accuracy = 97.88%
Cross Validation Accuracy = 97.98%
Cross Validation Accuracy = 98.08%
Cross Validation Accuracy = 98.06%
Cross Validation Accuracy = 98.02%
Cross Validation Accuracy = 97.76%
Cross Validation Accuracy = 97.92%
Cross Validation Accuracy = 98%
Cross Validation Accuracy = 96.78%
Cross Validation Accuracy = 97.54%
Cross Validation Accuracy = 97.92%
Cross Validation Accuracy 

Cross Validation Accuracy = 98.04%
Cross Validation Accuracy = 98.1%
Cross Validation Accuracy = 96.44%
Cross Validation Accuracy = 97.84%
Cross Validation Accuracy = 97.96%
Cross Validation Accuracy = 98%
Cross Validation Accuracy = 98%
Cross Validation Accuracy = 97.96%
Cross Validation Accuracy = 97.5%
Cross Validation Accuracy = 98%
Cross Validation Accuracy = 98.04%
Cross Validation Accuracy = 96.6%
Cross Validation Accuracy = 97.48%
Cross Validation Accuracy = 97.8%
Cross Validation Accuracy = 98%
Cross Validation Accuracy = 98.1%
Cross Validation Accuracy = 98.02%
Cross Validation Accuracy = 97.5%
Cross Validation Accuracy = 97.8%
Cross Validation Accuracy = 98.04%
Cross Validation Accuracy = 96.5%
Cross Validation Accuracy = 97.1%
Cross Validation Accuracy = 97.34%
Cross Validation Accuracy = 98.06%
Cross Validation Accuracy = 98.2%
Cross Validation Accuracy = 98.18%
Cross Validation Accuracy = 97.5%
Cross Validation Accuracy = 97.72%
Cross Validation Accuracy = 97.82%
Cross Va

KeyboardInterrupt: 

In [18]:
from subprocess import *
begin_level = round(max(x[2] for x in RBF_res)) - 3
step_size = 0.5
gnuplot = Popen('/usr/local/bin/gnuplot',stdin = PIPE,stdout=PIPE,stderr=PIPE).stdin
gnuplot.write(b"set term png transparent small linewidth 2 medium enhanced\n")
gnuplot.write("set output \"OUT.png\"\n".encode())
gnuplot.write(b"set xlabel \"log2(C)\"\n")
gnuplot.write(b"set ylabel \"log2(gamma)\"\n")
gnuplot.write("set xrange [{0}:{1}]\n".format(c_begin,c_end).encode())
gnuplot.write("set yrange [{0}:{1}]\n".format(g_begin,g_end).encode())
gnuplot.write(b"set contour\n")
gnuplot.write("set cntrparam levels incremental {0},{1},100\n".format(begin_level,step_size).encode())
gnuplot.write(b"unset surface\n")
gnuplot.write(b"unset ztics\n")
gnuplot.write(b"set view 0,0\n")
gnuplot.write("set title \"HW5 - handwritten digit\"\n".encode())
gnuplot.write(b"unset label\n")
gnuplot.write("set label \"Best log2(C) = {0}  log2(gamma) = {1}  accuracy = {2}%\" \
  at screen 0.5,0.85 center\n". \
  format(best_params[2][0], best_params[2][1], best_acc[2]).encode())
gnuplot.write("set label \"C = {0}  gamma = {1}\""
  " at screen 0.5,0.8 center\n".format(2**best_params[2][0], 2**best_params[2][1]).encode())
gnuplot.write(b"set key at screen 0.9,0.9\n")
gnuplot.write(b"splot \"-\" with lines\n")

RBF_res.sort(key = lambda x:(x[0], -x[1]))

prevc = RBF_res[0][0]
for line in RBF_res:
    if prevc != line[0]:
        gnuplot.write(b"\n")
        prevc = line[0]
    gnuplot.write("{0[0]} {0[1]} {0[2]}\n".format(line).encode())
gnuplot.write(b"e\n")
gnuplot.write(b"\n") # force gnuplot back to prompt when term set failure
gnuplot.flush()

In [19]:
best_params

[0.03125, (0.0625, 2, 2, 0), (0.0625, 2)]

In [20]:
best_acc

[96.94, 98.26, 97.88]